In [46]:
import os
import h3
import dotenv
import psycopg2
import json
import itertools
import pandas as pd
from h3_transformation import H3Transformation

## PostgreSQL

In [2]:
PG_CONF = {
    "host": os.environ.get("POSTGRES_HOST", None),
    "port": os.environ.get("POSTGRES_PORT", None),
    "user": os.environ.get("POSTGRES_USERNAME", None),
    "password": os.environ.get("POSTGRES_PASSWORD", None),
    "database": "spatial_dwh"
}

PG_CONF

{'host': '10.147.19.231',
 'port': '5432',
 'user': 'quan.bui',
 'password': 'P@ssw0rd123',
 'database': 'spatial_dwh'}

In [3]:
table = "vietnam_border"
limit = 100

conn = None
try:
    # Connect to database
    conn = psycopg2.connect(**PG_CONF)
    print(f'Connection established to database: "{PG_CONF["database"]}".')
    
    # Query
    cur = conn.cursor()
    cur.execute(
        f"""
        SELECT 
            country_id, 
            country_name, 
            ST_AsGeoJSON(geometry) AS geojson, 
            CONCAT_WS(',', ST_X(centroid), ST_Y(centroid)) AS centroid
        FROM {table}
        """
    )
    records = cur.fetchall()
    
    # Pandas
    records_df = pd.DataFrame(records, columns=[desc[0] for desc in cur.description])
    display(records_df.head())
    
except(Exception, psycopg2.DatabaseError) as error:
    print(error)
    

Connection established to database: "spatial_dwh".


,country_id,country_name,geojson,centroid
0,1,Vietnam,"{""type"":""MultiPolygon"",""coordinates"":[[[[113.8...","106.4063821609223,16.57755915233502"


In [4]:
# Extract data from PostgreSQL
(vn_centroid_lng, vn_centroid_lat) = records_df["centroid"].values[0].split(',')
vn_multipolygon_geojson = json.loads(records_df["geojson"].values[0])

# Transform MultiPolygon GeoJSON into several Polygon GeoJSONs
vn_polygon_geojsons = []
for polygon in vn_multipolygon_geojson["coordinates"]:
    vn_polygon_geojson = {
        "type": "Polygon",
        "coordinates": polygon
    }
    
    vn_polygon_geojsons.append(vn_polygon_geojson)

print(vn_polygon_geojsons[-1])

{'type': 'Polygon', 'coordinates': [[[107.39292908, 21.03173828], [107.38712311, 21.0298233], [107.38407135, 21.0219307], [107.38005829, 21.01855278], [107.38323975, 21.03806877], [107.38842011, 21.03846359], [107.3915329, 21.0358448], [107.39292908, 21.03173828]]]}


## H3

In [51]:
resolution = 9 # avg_area=0.0021496 km^2, avg_inradius=0.024910561 km

h_cell_sets = []
for polygon_geojson in vn_polygon_geojsons:
    h_cell_sets.append(h3.polyfill(geojson=polygon_geojson, res=resolution, geo_json_conformant=True))
    
# polyfill_geojson = H3Transformation.h3_to_geojson(h_cells)
# print(polyfill_geojson)

In [52]:
h_cells = set().union(*h_cell_sets)

In [53]:
h_cells_subset = set(itertools.islice(h_cells, 1500))

polyfill_geojson = H3Transformation.h3_to_geojson(h_cells_subset)
print(polyfill_geojson)

{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[103.32060368531722, 20.944543599825263], [103.32012025778523, 20.946403536344963], [103.31814500083324, 20.946984881478144], [103.31665318172894, 20.94570627928845], [103.31713663630465, 20.94384633119123], [103.31911188294173, 20.943264996860773], [103.32060368531722, 20.944543599825263]]]}, "properties": {"h3_idx": "89414a34027ffff", "is_valid_cell": true, "center": [20.945124949705903, 103.31862844533276], "area_m2": 117533.86242348608, "resolution": 9}}, {"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[102.41221954827618, 22.388081048605898], [102.41172649573355, 22.389923590324695], [102.40973306957638, 22.390494631869625], [102.40823270761574, 22.38922312298251], [102.40872578666784, 22.38738057104372], [102.41071920117196, 22.386809538211544], [102.41221954827618, 22.388081048605898]]]}, "properties": {"h3_idx": "8940608a587ffff", "is_valid_cell":